In [1]:
import json
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import nltk

In [2]:
# import relevant LMs package

import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score


In [3]:
# cheak to use gpu or cpu
if torch.cuda.is_available():
    print("Use gpu")
    device = torch.device("cuda")
else:
    print("Use cpu")
    device = torch.device("cpu")

Use gpu


In [4]:
s = 0
t = 10000
a = (1-np.random.rand(t,30000))*9
np.shape(a)

(10000, 30000)

In [5]:
m = np.mean(a,axis=1)
m
np.sum(np.abs(m-4.5)<1/12)/t

1.0

## KG

In [6]:
# KGs

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [7]:
import csv
# import wikipedia sentences
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

candidate_sentences = pd.read_csv("/Users/wz/IdeaProjects/context-aware-embedding/Resources/code/input/wiki_sentences_v2.csv")
candidate_sentences.shape

Files in 'C:\\Users\\wz\\Desktop\\context-aware-embedding-master\\Resources\\code': ['.DS_Store', '.ipynb_checkpoints', 'accuracy.py', 'Fun online example.ipynb', 'input', 'Messing around.ipynb', 'Prototype 0.0.ipynb', 'Prototype0.1.ipynb', 'query_trl.ipynb', 'test_trainer', 'trl', 'wandb', 'yago2', '__pycache__']


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wz/IdeaProjects/context-aware-embedding/Resources/code/input/wiki_sentences_v2.csv'

In [ ]:
candidate_sentences['sentence'].sample(5)

In [ ]:
doc = nlp("the drawdown process is governed by astm standard d823")

for tok in doc:
    print(tok.text, "...", tok.dep_)

In [ ]:
def get_entities(sent):
## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [ ]:
get_entities("the film had 200 patents")

In [ ]:
entity_pairs = []

for i in tqdm(candidate_sentences["sentence"]):
    entity_pairs.append(get_entities(i))

In [ ]:
entity_pairs[10:20]

In [ ]:
def get_relation(sent):
    doc = nlp(sent)

    # Matcher class object 
    matcher = Matcher(nlp.vocab)

    #define the pattern 
    pattern = [{'DEP':'ROOT'}, 
                {'DEP':'prep','OP':"?"},
                {'DEP':'agent','OP':"?"},  
                {'POS':'ADJ','OP':"?"}] 

    matcher.add("matching_1",[pattern],on_match=None ) 

    matches = matcher(doc)
    k = len(matches) -1

    span = doc[matches[k][1]:matches[k][2]] 

    return(span.text)

In [ ]:
get_relation("John completed the task")
len(candidate_sentences["sentence"])

In [ ]:
#  256 th  和 2700 th sentence 有问题 跳过这句 
# for i in tqdm(candidate_sentences['sentence'][257:2000]):
#     get_relation(i)
    

In [ ]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['sentence'][270:2700]) ]

In [ ]:
pd.Series(relations).value_counts()[:50]

In [ ]:
# extract subject
source = [i[0] for i in entity_pairs][270:2700]

# extract object
target = [i[1] for i in entity_pairs][270:2700]

# print(len(source))
# print(len(target))
# print(len(relations))

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})



In [ ]:
kg_df

In [ ]:
def contains(source, target, relations):
    triple = np.array([source, target, relations])
    return (kg_df==triple).all(1).any()

contains("connie", "own",  "decides")

In [ ]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

### Extract from YAGO2

In [ ]:
colnames=['subject', 'predicate', 'object'] 
# YAGO = pd.read_csv("/Users/wz/IdeaProjects/context-aware-embedding/Resources/code/yago2/yago2core_facts.clean.notypes.tsv" , names=colnames, header=None,sep ='\t')
YAGO = pd.read_csv("yago2\\yago2core_facts.clean.notypes.tsv" , names=colnames, header=None,sep ='\t')
YAGO

In [ ]:
# remove useless lines
# 1-260: xxx http://www.w3.org/2000/01/rdf-schema <yago>
YAGO = YAGO.iloc[261:557]
# remove the <> sign
YAGO['subject'] = YAGO['subject'].apply(lambda x: x[1:-1])
YAGO['predicate'] = YAGO['predicate'].apply(lambda x: x[1:-1])
YAGO['object'] = YAGO['object'].apply(lambda x: x[1:-1])
YAGO

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

## LM

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

config = {
    "lm_name": "lvwerra/gpt2-imdb",
    "ref_lm_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/distilbert-imdb",
    "tk_name": "gpt2",
    "steps": 320,
    "batch_size": 32,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "txt_in_len": 2,
    "txt_out_len": 1,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "seed": 1,
}

np.random.seed(config['seed'])


In [ ]:
# trails
wandb.init(name='trails #1', project='gpt2-ctrl', config=config)

### Load pre-trained GPT2 language models

In [ ]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(config['lm_name'])
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(config['ref_lm_name'])
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(config['tk_name'])

In [ ]:
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)

In [ ]:
wandb.watch(gpt2_model, log='all')

### Load BERT classifier

In [ ]:
device = "cuda:0"

sentiment_model = AutoModelForSequenceClassification.from_pretrained(config["cls_model_name"])
sentiment_tokenizer = AutoTokenizer.from_pretrained(config["cls_model_name"])

# model = model.to(device)

In [ ]:
text = 'this movie was really bad!!'
output = sentiment_model.forward(sentiment_tokenizer.encode(text, return_tensors="pt"))
output

In [ ]:
output[0][0, 1]

### Next Token Prediction with GPT2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


print("\nBegin next-word using HF GPT-2 demo ")

toker = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
seq = "connie own"
print("\nInput sequence: ")
print(seq)

inpts = toker(seq, return_tensors="pt")
print("\nTokenized input data structure: ")
print(inpts)

inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
print("\nToken IDs and their words: ")
for id in inpt_ids[0]:
    word = toker.decode(id)
    print(id, word)

    
with torch.no_grad():
    logits = model(**inpts).logits[:, -1, :]

print("\nAll logits for next word: ")
print(logits)
print(logits.shape)

pred_id = torch.argmax(logits).item()
print("\nPredicted token ID of next word: ")
print(pred_id)

pred_word = toker.decode(pred_id)
print("\nPredicted next word for sequence: ")
print(pred_word)


In [ ]:
def GPT_pred(seq):
    inpts = toker(seq, return_tensors="pt")
    inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
    with torch.no_grad():
        logits = model(**inpts).logits[:, -1, :]
    pred_id = torch.argmax(logits).item()
    pred_word = toker.decode(pred_id)
    return logits, pred_id, pred_word
    

In [ ]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i]=='[false]':
            logit[i] = -logit[i]
        elif task[i]=='[true]':
            pass
        else:
            raise ValueError('task has to be in [0, 1, 2]!')
    return logit

In [ ]:
df = kg_df[:1000]

# device = torch.device("cpu")
#model(img_test.unsqueeze(0).cuda()).detach().cpu().clone().numpy()

# modify query as object + relation
# e.g connie owns 
df['query'] = df.apply(
    lambda row: row.source + ' ' + row.edge, axis=1)
# adjust format in list 
df['query'] = [df['query'][i].split()[:2] for i in range(1000)]
df['query'] = df['query'].apply(lambda x: ' '.join(x))

#special token
df['tokens'] = df['query'].apply(lambda x: gpt2_tokenizer.encode(' '+str(x), return_tensors="pt").to(device))
df['tokens'] = df['tokens'].apply(lambda x: x[0])
df['tokens'] = df['tokens'].apply(lambda x: x[:2])

# dropping invalid length in token column
drop_row_index = []
for i in range(1000):

    if len(df['tokens'].loc[i]) < 2:
        #df['tokens'].loc[i] = df['tokens'].loc[0].cuda().detach().cpu().clone().numpy()
        drop_row_index.append(i)
print(drop_row_index)
df = df.drop(df.index[drop_row_index])
        

        
# df.loc[(len(df['tokens']) < 2), 'tokens'] = df['tokens'].loc[0]
#df['tokens'] = df['tokens'].apply(lambda x: df['tokens'].loc[0] if len(x) < 2)



ctrl_str = ['[false]', '[true]']
ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [ ]:
df

### PPO & Optimization

In [ ]:
# device = torch.device("cuda")
# device = "cuda:0"
#input = input.to(device)
#         model = model.to(device)

a = [0,0,1,0]
a = [i + random.random() for i in a ]
a

In [ ]:
### KG check to modify reward value 

# given a range i.e. i*fbs to (i+1)*fbs
# should output a tensor([ 0, 0,  1, 1], device='cuda:0')

# pred is the reponse in batch_response
# query is the query_list

def KG_check(lower, upper):
    tensor_array = []
    for i in range(lower,upper):
        # e.g. dark knight not
#         print('lower is ',lower )
#         print('upper is ',upper )
#         print("i is ", i)
#         print(len(query_list))
#         print(len(batch_response))

        # bugs here todo query_list[i].split() could be one element only
        source = query_list[i].split()[0]
        target = batch_response[i]
        if len(query_list[i].split()) <2:
            
            relation = 'anything'
        else:
            relation = query_list[i].split()[1]
        
#         print(i)
#         print("source is ",source)
#         print("target is ",target)
#         print("relation is ",relation)
        if contains(source, target, relation):
            tensor_array.append(1)
        else:
            tensor_array.append(0)
            
    # rand modular Todo: Fix this
    c = random.randint(0, 3)
    tensor_array[c]=1
    
    # add variant 
    tensor_array = [i + random.random() for i in tensor_array ]
    return torch.cuda.LongTensor(tensor_array)

# KG_check(28,32)

In [ ]:
# debug

# df_batch

In [ ]:
# debug

# df_results = pd.DataFrame(game_data)
# df_results
#query_list

In [ ]:
# for testing

# batch_size: 32
# forward_batch_size: 4 

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset and annotate tasks
    df_batch = df.sample(config['batch_size'])
    task_list = choices(ctrl_str, k=config['batch_size'])
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])
    query_list = df_batch['query'].tolist()
    game_data['query'] = [t+q for t,q in zip(task_list, query_list)]
    
    
    #print(task_tensors)
    query_tensors = torch.stack((df_batch['tokens'].tolist()))
    #print("before ",query_tensors)
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    
    #### get response from gpt2
    t = time.time()
    response_tensors_array = []
    for i in range(int(config['batch_size']/fbs)):
        #print(int(config['batch_size']/fbs))
        # 8 loops in total
        # 0-4, 5-8, 9-12...
        
        # this is troublesome 
        # error: probability tensor contains either `inf`, `nan` or element < 0
        #  next_token = torch.multinomial(probs, num_samples=1).squeeze(1) in gpt2.py function respond_to_batch
        
        # response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],txt_len=config['txt_out_len'])
        # we want a response of size [i*fbs:(i+1)*fbs]*1
        response_tensors = []
        for i in range(i*fbs, (i+1)*fbs):
            seq = query_list[i]
            inpts = toker(seq, return_tensors="pt")
            inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
            for id in inpt_ids[0]:
                word = toker.decode(id)

            with torch.no_grad():
                logits = model(**inpts).logits[:, -1, :]

            pred_id = torch.argmax(logits)
        
            response = pred_id
            
            response_tensors.append(response.item())
        response_tensors = torch.cuda.LongTensor(response_tensors)
        response_tensors_array.append(response_tensors)
   
    # response_tensors.size = 32 * 1  
    response_tensors = torch.cat(response_tensors_array,axis = -1 )
    response_tensors  = torch.reshape(response_tensors, (-1, 1)) # force size of response_tensors.size = 32 * 1  rather that 32 length list
    #print(response_tensors)
    #print([gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])])
    batch_response = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])] # list of length 32
    game_data['response'] = batch_response
    timing['time/get_response'] = time.time()-t
    #print( game_data['response'])

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(query_list, game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment'] = time.time()-t
        
    #### get sentiment score
    t = time.time()
    pos_logits = []
    for i in range(int(config['batch_size']/fbs)):
        input = {'input_ids':      sentiment_inputs[i*fbs:(i+1)*fbs],
                  'attention_mask': attention_masks[i*fbs:(i+1)*fbs],
                  'labels':         None
                 }      
        #res = sentiment_model.forward(**input)[0][:, 1].detach()
        
        device = torch.device("cuda")
        sentiment_model = sentiment_model.to(device)
        res = sentiment_model.forward(sentiment_inputs[i*fbs:(i+1)*fbs],
                                      attention_masks[i*fbs:(i+1)*fbs])[0][:, 1].detach()
#         print(res)
#         print(res.size())

        # modify the rewards based on the KG
#         print('lower is ',i*fbs )
#         print('upper is ',(i+1)*fbs )
        res = KG_check(i*fbs, (i+1)*fbs)

        pos_logits.append(res)
    
    rewards = pos_logit_to_reward(torch.cat(pos_logits), task_list)
    timing['time/get_sentiment_preds'] = time.time()-t


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # todo fix this 
    # enforce nan to zero for wandb to run
    stats = np.nan_to_num(stats)
    #print(stats)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    # comment to remove the nan issues
    #logs.update(stats)
    
    # only support for float tensor
    rewards = rewards.type(torch.FloatTensor)
    
    logs['env/reward_mean'] = torch.mean((rewards)).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    for ctrl_s in ctrl_str:
        key = 'env/reward_'+ctrl_s.strip('[]')
        logs[key] = np.mean([r for r, t in zip(logs['env/reward_dist'], task_list) if t==ctrl_s])
#     WandbCallback(log=None)
    wandb.log(logs)

In [ ]:
# debug


#     cuda0 = torch.device('cuda:0')

#     for i in range(config['batch_size']):
#         current_row = df_batch['tokens'].iloc[i]
    
#         if type(current_row) is np.ndarray:
#             #print('before ', current_row[0])
#             #print(i)
#             val = [current_row[0], current_row[1]]
#             df_batch['tokens'].iloc[i] = list((torch.cuda.LongTensor([val[0]]),torch.cuda.LongTensor([val[1]])))
#             print(i)
#             print(df_batch['tokens'].iloc[i])

In [ ]:
# i = [6096, 9000]
# list((torch.cuda.LongTensor(i[0]),torch.cuda.LongTensor(i[1])))
# #torch.cpu.as_tensor(i)

### Example

In [ ]:
#### get a batch from the dataset
bs = 32
game_data = dict()
df_batch = df.sample(bs)
query_list = df_batch['query'].tolist()
game_data['query'] = query_list
for ctrl in ctrl_str:
    task_list = [ctrl] * bs
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])

    query_tensors = torch.stack(df_batch['tokens'].tolist())
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    #### get response from gpt2 and gpt2_ref
    response_tensors  = respond_to_batch(gpt2_model, query_tensors, txt_len=config['txt_out_len'])
    game_data['response ' + ctrl] = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(bs)]

    #### sentiment analysis of query/response pairs before/after
    texts = [q + r for q,r in zip(game_data['query'], game_data['response ' + ctrl])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
    game_data['rewards ' + ctrl] = pos_logit_to_reward(rewards, task_list).cpu().numpy()

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

In [ ]:
# use BERT tokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# call tokenize.encode function to convert sentences

# # original sentence(X)
# print(' Original sentence: ', X[0])

# # tokenized sentence(X)
# print('Tokenized sentence: ', bert_tokenizer.tokenize(X[0]))

# # map sentence to token IDs
# print('Mapping to IDs: ', bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(X[0])))

# add [CLS] [SEP] [PAD] token as required 
# The encode funtion will 1. split into tokens 2. add [CLS] and [SEP] 3. map tokens to IDs
input_id = []
for s in X:
    encoded  = bert_tokenizer.encode(s,  # the sentence for encoding
                                     add_special_tokens = True,  # [CLS] and [SEP]
                                     max_length = 200, # maximum length of sentence
                                     pad_to_max_length = True, # padding length
                                     return_tensors = 'pt' # return type is tensor
                                    )
    input_id.append(encoded)


input_id = torch.cat(input_id, dim=0)
label_y = torch.tensor(Y) 

# set epoch and batch_size

epoch = 4
batch_size = 32

# split data for training and validation
data = TensorDataset(input_id,label_y)
data_len = len(data)
train, validation = random_split(data, [int(0.8*data_len), data_len - int(0.8*data_len)])

# create DataLoader for both training set and valiadation set
train_dl = DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(validation, batch_size = batch_size)



In [ ]:
# use BertForSequenceClassification as our model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                      num_labels=4, 
                                                      output_attentions=False, # no attention weight
                                                      output_hidden_states=False # not need to return all hidden state )
                                                     )
# use cpu to train
model.cpu()

# optimizer and lr schedule

# convention is to use AdamW where W is weight decay fix from huggingface
optim = AdamW(model.parameters(), lr = 2e-5)
scheduler = get_linear_schedule_with_warmup(optim, 
                                           num_warmup_steps = 0, # default
                                           num_training_steps = len(train_dl)*epoch)

# define accuracy by accuracy_score

def accuracy(pred_label, y):
    pred_label = np.argmax(pred_label, axis =1).flatten()
    return accuracy_score(pred_label, y.flatten())



In [ ]:
# set the seed value to make sure the return are determistic

seed_value = 488
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.backends.cudnn.deterministic = True

device = torch.device('cpu')


# training phase

for epc in range(epoch):
    # accumulated loss for training and validation
    training_loss = 0
    validation_loss = 0
    evaluation_acc = 0
    
    # training mode
    model.train()
    for step, batch in enumerate(train_dl):
        model.zero_grad()
        # each batch has 3 tensors: 1. input id 2. attention mask 3. label
        # e.g. batch[i].to(device)
        # follow the documentation on https://huggingface.co/transformers
        input_id = batch[0].to(device)
        att_mask = (batch[0]>0).to(device)
        input_label = batch[1].to(device)
        output = model(input_id, token_type_ids=None, attention_mask=att_mask, labels=input_label)
        loss = output[0]
        print("training batch ",step," with the loss of ", loss.item())
        training_loss += loss.item()
        # backprop
        loss.backward()
        # clip the gradient to 1.0, prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # update parameter
        optim.step() 
        # update lr
        scheduler.step()
        
    # eval mode
    model.eval()
    for step, batch in enumerate(val_dl):
        # indicate the model not to compute gradients !
        with torch.no_grad():
            input_id = batch[0].to(device)
            att_mask = (batch[0]>0).to(device)
            input_label = batch[1].to(device)
            output = model(input_id, token_type_ids=None, attention_mask=att_mask, labels=input_label)
            loss = output[0] 
            print("val batch ",step," with the loss of ", loss.item())
            validation_loss += loss.item()
            pred = output[1].detach().cpu().numpy()
            ground_truth_label = batch[1].to('cpu').numpy()
            evaluation_acc += accuracy(pred, ground_truth_label)
            
    print('Validation loss: ', validation_loss / len(val_dl))
    print('Overall accuracy: ', evaluation_acc / len(val_dl))

print('Training finished')

In [ ]:
# testing
# predicate test label

# add [CLS] [SEP] [PAD] token as required
input_id_test = []
for s in Xt:
    encoded  = bert_tokenizer.encode(s, add_special_tokens = True, max_length = 200, 
                                pad_to_max_length = True, return_tensors = 'pt')
    input_id_test.append(encoded)
    
input_id_test = torch.cat(input_id_test, dim=0)
test_data = TensorDataset(input_id_test)
test_dl = DataLoader(test_data, batch_size=batch_size)

model.eval()

pred_label = []
for step, (batch,) in enumerate(test_dl):
    if step % 10 == 0:
        print("processed with batch ", step)
    with torch.no_grad():
        output = model(batch.to(device), token_type_ids=None, attention_mask=(batch>0).to(device))
        pred = output[0].detach().cpu().numpy()
        pred_label.append(pred)
print('Test predicting finished')
final_preds = np.concatenate(pred_label, axis=0)



In [ ]:
# convert probabilites into labels
Y_test_pred = final_preds

Y_pred = np.zeros((Y_test_pred.shape[0],1))

for i in range (Y_test_pred.shape[0]):
    array = Y_test_pred[i]
    
    max_val = np.max(array)
    
    index_max = np.where(array == max_val)
    Y_pred[i] = index_max[0][0]
    
    
print(Y_pred)

In [ ]:
# write out the csv file
# first column is the id, it is the index to the list of test examples
# second column is the predction as an integer
fout = open("out.csv", "w")
fout.write("Id,Y\n")
for i, line in enumerate(Y_pred): # Y_test_pred is in the same order as the test data
    fout.write("%d,%d\n" % (i, line))
fout.close()


In [ ]:
### REFERENCE

# https://www.kaggle.com/hassanamin/bert-pytorch-cola-classification
# https://www.cnblogs.com/tangjianwei/p/13334327.html